In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import os
import re

# Funzione per estrarre e convertire i valori dell'ottava colonna in liste di float
def get_eight_column_values(df, type_val, module_val, name_val):
    filtered_df = df[(df['type'] == type_val) & (df['module'] == module_val) & (df['name'] == name_val)]
    if filtered_df.empty:
        return None
    values = filtered_df.iloc[:, 7].str.split().apply(lambda x: list(map(float, x)))
    return values.explode().astype(float).values

# Funzione per estrarre e convertire i valori della settima colonna in liste di float
def get_seventh_column_values(df, type_val, module_val, name_val):
    filtered_df = df[(df['type'] == type_val) & (df['module'] == module_val) & (df['name'] == name_val)]
    if filtered_df.empty:
        return None
    values = filtered_df.iloc[:, 6].str.split().apply(lambda x: list(map(float, x)))
    return values.explode().astype(float).values

# Funzione per calcolare l'intervallo di confidenza t-student
def t_student_confidence_interval(data, confidence=0.95):
    mean = np.mean(data)
    sem = stats.sem(data)  # Standard error of the mean
    margin = sem * stats.t.ppf((1 + confidence) / 2, len(data) - 1)
    return mean, mean - margin, mean + margin

# Funzione per calcolare l'intervallo di confidenza naive
def naive_confidence_interval(data, confidence=0.95):
    mean = np.mean(data)
    margin = mean * (1 - confidence)
    return mean, mean - margin, mean + margin

# Funzione per calcolare l'intervallo di confidenza normale
def normal_confidence_interval(data, confidence=0.95):
    mean = np.mean(data)
    std_error = stats.sem(data)  # Standard error of the mean
    z_value = stats.norm.ppf(1 - (1 - confidence) / 2)
    margin = z_value * std_error
    return mean, mean - margin, mean + margin

# Funzione per rimuovere '_vec' dalla fine del nome del file
def remove_vec_suffix(file_name):
    if file_name.endswith('_vec.csv'):
        return file_name[:-8]
    else:
        return file_name

# Funzione per ottenere il numero finale dal nome del file
def get_file_number(file_name):
    match = re.search(r'\d+$', file_name)
    if match:
        return int(match.group())
    else:
        return 0  # Se non viene trovato nessun numero, restituisci 0

# Funzione per analizzare un singolo file CSV
def analyze_csv(file_path, ignored_files, results):
    df = pd.read_csv(file_path)
    description = df[df['attrname'] == 'description']['attrvalue'].values
    description = description[0] if len(description) > 0 else "N/A"

    is_N = any("N=1" in val for val in df[df['attrname'] == 'description']['attrvalue'].values)

    pDistribution = get_eight_column_values(df, "vector", "TandemQueueSystem.Server", "pDistribution")
    vDistribution = get_eight_column_values(df, "vector", "TandemQueueSystem.Server", "vDistribution")
    lifeTime = get_eight_column_values(df, "vector", "TandemQueueSystem.sink", "lifeTime:vector")
    lifeTime_arrival = get_seventh_column_values(df, "vector", "TandemQueueSystem.sink", "lifeTime:vector")
    lifeTime_change_Q1 = get_seventh_column_values(df, "vector", "TandemQueueSystem.Q1", "queueLength:vector")
    lifeTime_change_Q2 = get_seventh_column_values(df, "vector", "TandemQueueSystem.Q2", "queueLength:vector")

    if pDistribution is None or vDistribution is None or lifeTime is None:
        ignored_files.append((remove_vec_suffix(os.path.basename(file_path)), description))
        return

    min_length = min(len(pDistribution), len(vDistribution), len(lifeTime))
    pDistribution = pDistribution[:min_length]
    vDistribution = vDistribution[:min_length]
    lifeTime = lifeTime[:min_length]

    queueLength_Q1 = get_eight_column_values(df, "vector", "TandemQueueSystem.Q1", "queueLength:vector")
    queueLength_Q2 = get_eight_column_values(df, "vector", "TandemQueueSystem.Q2", "queueLength:vector")

    if is_N:
        queueLength_Q2 = np.zeros(len(queueLength_Q1))
    else: 
        if queueLength_Q1 is None or queueLength_Q2 is None:
            ignored_files.append((remove_vec_suffix(os.path.basename(file_path)), description))
            return

    Cw = 1  # Definisci il valore di Cw

    conf_levels = [0.95, 0.90, 0.85, 0.80]
    lifeTime_stats_t_student = {}
    U_values_stats_t_student = {}
    utilization_Q1_stats_t_student = {}
    utilization_Q2_stats_t_student = {}

    lifeTime_stats_naive = {}
    U_values_stats_naive = {}
    utilization_Q1_stats_naive = {}
    utilization_Q2_stats_naive = {}
    
    lifeTime_stats_normal = {}
    U_values_stats_normal = {}
    utilization_Q1_stats_normal = {}
    utilization_Q2_stats_normal = {}

    for conf in conf_levels:
        # Calcoli t-student
        mean_W_t, ci_low_W_t, ci_high_W_t = t_student_confidence_interval(lifeTime, confidence=conf)
        lifeTime_stats_t_student[conf] = (mean_W_t, ci_low_W_t, ci_high_W_t)
        
        U_values = vDistribution - pDistribution - (Cw * lifeTime)
        mean_U_t, ci_low_U_t, ci_high_U_t = t_student_confidence_interval(U_values, confidence=conf)
        U_values_stats_t_student[conf] = (mean_U_t, ci_low_U_t, ci_high_U_t)

        mean_utilization_Q1_t, ci_low_utilization_Q1_t, ci_high_utilization_Q1_t = t_student_confidence_interval(queueLength_Q1 / np.sum(queueLength_Q1), confidence=conf)
        utilization_Q1_stats_t_student[conf] = (mean_utilization_Q1_t, ci_low_utilization_Q1_t, ci_high_utilization_Q1_t)

        mean_utilization_Q2_t, ci_low_utilization_Q2_t, ci_high_utilization_Q2_t = t_student_confidence_interval(queueLength_Q2 / np.sum(queueLength_Q2), confidence=conf)
        utilization_Q2_stats_t_student[conf] = (mean_utilization_Q2_t, ci_low_utilization_Q2_t, ci_high_utilization_Q2_t)

        # Calcoli naive
        mean_W_n, ci_low_W_n, ci_high_W_n = naive_confidence_interval(lifeTime, confidence=conf)
        lifeTime_stats_naive[conf] = (mean_W_n, ci_low_W_n, ci_high_W_n)
        
        mean_U_n, ci_low_U_n, ci_high_U_n = naive_confidence_interval(U_values, confidence=conf)
        U_values_stats_naive[conf] = (mean_U_n, ci_low_U_n, ci_high_U_n)

        mean_utilization_Q1_n, ci_low_utilization_Q1_n, ci_high_utilization_Q1_n = naive_confidence_interval(queueLength_Q1 / np.sum(queueLength_Q1), confidence=conf)
        utilization_Q1_stats_naive[conf] = (mean_utilization_Q1_n, ci_low_utilization_Q1_n, ci_high_utilization_Q1_n)

        mean_utilization_Q2_n, ci_low_utilization_Q2_n, ci_high_utilization_Q2_n = naive_confidence_interval(queueLength_Q2 / np.sum(queueLength_Q2), confidence=conf)
        utilization_Q2_stats_naive[conf] = (mean_utilization_Q2_n, ci_low_utilization_Q2_n, ci_high_utilization_Q2_n)

        # Calcoli normale
        mean_W_norm, ci_low_W_norm, ci_high_W_norm = normal_confidence_interval(lifeTime, confidence=conf)
        lifeTime_stats_normal[conf] = (mean_W_norm, ci_low_W_norm, ci_high_W_norm)
        
        mean_U_norm, ci_low_U_norm, ci_high_U_norm = normal_confidence_interval(U_values, confidence=conf)
        U_values_stats_normal[conf] = (mean_U_norm, ci_low_U_norm, ci_high_U_norm)

        mean_utilization_Q1_norm, ci_low_utilization_Q1_norm, ci_high_utilization_Q1_norm = normal_confidence_interval(queueLength_Q1 / np.sum(queueLength_Q1), confidence=conf)
        utilization_Q1_stats_normal[conf] = (mean_utilization_Q1_norm, ci_low_utilization_Q1_norm, ci_high_utilization_Q1_norm)

        mean_utilization_Q2_norm, ci_low_utilization_Q2_norm, ci_high_utilization_Q2_norm = normal_confidence_interval(queueLength_Q2 / np.sum(queueLength_Q2), confidence=conf)
        utilization_Q2_stats_normal[conf] = (mean_utilization_Q2_norm, ci_low_utilization_Q2_norm, ci_high_utilization_Q2_norm)

    max_W = np.max(lifeTime)
    min_W = np.min(lifeTime)

    results.append({
        "File": remove_vec_suffix(os.path.basename(file_path)),
        "File_Number": get_file_number(remove_vec_suffix(os.path.basename(file_path))),
        "Description": description,
        "max_W": max_W,
        "min_W": min_W,
        **{f"mean_W_t_student_{int(conf*100)}": lifeTime_stats_t_student[conf][0] for conf in conf_levels},
        **{f"ci_low_W_t_student_{int(conf*100)}": lifeTime_stats_t_student[conf][1] for conf in conf_levels},
        **{f"ci_high_W_t_student_{int(conf*100)}": lifeTime_stats_t_student[conf][2] for conf in conf_levels},
        **{f"mean_U_t_student_{int(conf*100)}": U_values_stats_t_student[conf][0] for conf in conf_levels},
        **{f"ci_low_U_t_student_{int(conf*100)}": U_values_stats_t_student[conf][1] for conf in conf_levels},
        **{f"ci_high_U_t_student_{int(conf*100)}": U_values_stats_t_student[conf][2] for conf in conf_levels},
        **{f"mean_utilization_Q1_t_student_{int(conf*100)}": utilization_Q1_stats_t_student[conf][0] for conf in conf_levels},
        **{f"ci_low_utilization_Q1_t_student_{int(conf*100)}": utilization_Q1_stats_t_student[conf][1] for conf in conf_levels},
        **{f"ci_high_utilization_Q1_t_student_{int(conf*100)}": utilization_Q1_stats_t_student[conf][2] for conf in conf_levels},
        **{f"mean_utilization_Q2_t_student_{int(conf*100)}": utilization_Q2_stats_t_student[conf][0] for conf in conf_levels},
        **{f"ci_low_utilization_Q2_t_student_{int(conf*100)}": utilization_Q2_stats_t_student[conf][1] for conf in conf_levels},
        **{f"ci_high_utilization_Q2_t_student_{int(conf*100)}": utilization_Q2_stats_t_student[conf][2] for conf in conf_levels},
        **{f"mean_W_naive_{int(conf*100)}": lifeTime_stats_naive[conf][0] for conf in conf_levels},
        **{f"ci_low_W_naive_{int(conf*100)}": lifeTime_stats_naive[conf][1] for conf in conf_levels},
        **{f"ci_high_W_naive_{int(conf*100)}": lifeTime_stats_naive[conf][2] for conf in conf_levels},
        **{f"mean_U_naive_{int(conf*100)}": U_values_stats_naive[conf][0] for conf in conf_levels},
        **{f"ci_low_U_naive_{int(conf*100)}": U_values_stats_naive[conf][1] for conf in conf_levels},
        **{f"ci_high_U_naive_{int(conf*100)}": U_values_stats_naive[conf][2] for conf in conf_levels},
        **{f"mean_utilization_Q1_naive_{int(conf*100)}": utilization_Q1_stats_naive[conf][0] for conf in conf_levels},
        **{f"ci_low_utilization_Q1_naive_{int(conf*100)}": utilization_Q1_stats_naive[conf][1] for conf in conf_levels},
        **{f"ci_high_utilization_Q1_naive_{int(conf*100)}": utilization_Q1_stats_naive[conf][2] for conf in conf_levels},
        **{f"mean_utilization_Q2_naive_{int(conf*100)}": utilization_Q2_stats_naive[conf][0] for conf in conf_levels},
        **{f"ci_low_utilization_Q2_naive_{int(conf*100)}": utilization_Q2_stats_naive[conf][1] for conf in conf_levels},
        **{f"ci_high_utilization_Q2_naive_{int(conf*100)}": utilization_Q2_stats_naive[conf][2] for conf in conf_levels},
        **{f"mean_W_normal_{int(conf*100)}": lifeTime_stats_normal[conf][0] for conf in conf_levels},
        **{f"ci_low_W_normal_{int(conf*100)}": lifeTime_stats_normal[conf][1] for conf in conf_levels},
        **{f"ci_high_W_normal_{int(conf*100)}": lifeTime_stats_normal[conf][2] for conf in conf_levels},
        **{f"mean_U_normal_{int(conf*100)}": U_values_stats_normal[conf][0] for conf in conf_levels},
        **{f"ci_low_U_normal_{int(conf*100)}": U_values_stats_normal[conf][1] for conf in conf_levels},
        **{f"ci_high_U_normal_{int(conf*100)}": U_values_stats_normal[conf][2] for conf in conf_levels},
        **{f"mean_utilization_Q1_normal_{int(conf*100)}": utilization_Q1_stats_normal[conf][0] for conf in conf_levels},
        **{f"ci_low_utilization_Q1_normal_{int(conf*100)}": utilization_Q1_stats_normal[conf][1] for conf in conf_levels},
        **{f"ci_high_utilization_Q1_normal_{int(conf*100)}": utilization_Q1_stats_normal[conf][2] for conf in conf_levels},
        **{f"mean_utilization_Q2_normal_{int(conf*100)}": utilization_Q2_stats_normal[conf][0] for conf in conf_levels},
        **{f"ci_low_utilization_Q2_normal_{int(conf*100)}": utilization_Q2_stats_normal[conf][1] for conf in conf_levels},
        **{f"ci_high_utilization_Q2_normal_{int(conf*100)}": utilization_Q2_stats_normal[conf][2] for conf in conf_levels},
        "mean_queueLength_Q1": np.mean(queueLength_Q1),
        "mean_queueLength_Q2": np.mean(queueLength_Q2),
        "lifeTime": ",".join(map(str, lifeTime)),
        "lifeTime_arrival": ",".join(map(str, lifeTime_arrival)),
        "queueLength_Q1": ",".join(map(str, queueLength_Q1)),
        "queueLength_Q2": ",".join(map(str, queueLength_Q2)),
        "lifeTime_change_Q1": ",".join(map(str, lifeTime_change_Q1)),
        "lifeTime_change_Q2": ",".join(map(str, lifeTime_change_Q2))
    })

# Funzione per analizzare tutti i file CSV in una directory
def analyze_directory(directory_path):
    ignored_files = []
    results = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.csv') and not file_name.endswith('_sca.csv'):
            file_path = os.path.join(directory_path, file_name)
            analyze_csv(file_path, ignored_files, results)
    
    if results:
        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values(by='File_Number')  # Ordina per numero del file
        results_df.drop(columns=['File_Number'], inplace=True)  # Rimuovi la colonna File_Number
        results_df.to_csv(os.path.join('./results_summary_convalidation_3.csv'), index=False)
    
    if ignored_files:
        ignored_files_sorted = sorted(ignored_files, key=lambda x: get_file_number(x[0]))  # Ordina gli ignorati per numero del file
        ignored_df = pd.DataFrame(ignored_files_sorted, columns=['File', 'Description'])
        ignored_df.to_csv(os.path.join('./ignored_files_convalidation_3.csv'), index=False)

# Esempio di utilizzo
directory_path = './results_CSV_convalidation_3'
analyze_directory(directory_path)


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

# Carica il DataFrame dal file CSV
df = pd.read_csv('./results_summary_convalidation_3.csv')

df = df[df["File"].str.contains("Config2")]

# Converti le colonne in liste di float
df["queueLength_Q1"] = df["queueLength_Q1"].apply(lambda x: list(map(lambda y: int(float(y)), x.split(','))))
df["queueLength_Q2"] = df["queueLength_Q2"].apply(lambda x: list(map(lambda y: int(float(y)), x.split(','))))
df["lifeTime_change_Q1"] = df["lifeTime_change_Q1"].apply(lambda x: list(map(float, x.split(','))))
df["lifeTime_change_Q2"] = df["lifeTime_change_Q2"].apply(lambda x: list(map(float, x.split(','))))

# Dizionario per accumulare i valori di lt_diff per ogni simulazione e numero di utenti
lt_diff_dict = {}

# Itera su tutte le righe del DataFrame filtrato
for index, row in df.iterrows():
    UsersQ1 = row["queueLength_Q1"]
    UsersQ2 = row["queueLength_Q2"]
    TimesQ1 = row["lifeTime_change_Q1"]
    TimesQ2 = row["lifeTime_change_Q2"]

    # Funzione per ottenere il numero di utenti in un determinato momento
    def get_user_count(times, users, time):
        for i, t in enumerate(times):
            if t == time:
                return users[i]
            elif t > time:
                return users[i-1]
        return users[-1]

    # Combiniamo e ordiniamo i tempi
    combined_times = sorted(set(TimesQ1 + TimesQ2))

    # Calcoliamo i valori combinati degli utenti
    UsersQ1_Q2 = []
    for time in combined_times:
        users_q1 = get_user_count(TimesQ1, UsersQ1, time)
        users_q2 = get_user_count(TimesQ2, UsersQ2, time)
        UsersQ1_Q2.append(users_q1 + users_q2)

    # Calcolo delle differenze di tempo (lt_diff) per questa simulazione
    sim_lt_diff = {}
    for i in range(1, len(combined_times)):
        lt_difference = combined_times[i] - combined_times[i-1]
        key = UsersQ1_Q2[i-1]
        if key not in sim_lt_diff:
            sim_lt_diff[key] = 0
        sim_lt_diff[key] += lt_difference

    # Aggiungiamo i lt_diff di questa simulazione al dizionario generale
    for key, value in sim_lt_diff.items():
        if key not in lt_diff_dict:
            lt_diff_dict[key] = []
        lt_diff_dict[key].append(value)

# Funzione per calcolare l'intervallo di confidenza di Student
def t_student_confidence_interval(data, confidence=0.90):
    mean = np.mean(data)
    sem = stats.sem(data)  # Standard error of the mean
    margin = sem * stats.t.ppf((1 + confidence) / 2, len(data) - 1)
    return mean, mean - margin, mean + margin

# Lista per accumulare i dati delle nuove righe
new_data = []

# Calcolo dell'intervallo di confidenza e creazione delle nuove righe
for key, lt_diffs in lt_diff_dict.items():
    mean, lower_bound, upper_bound = t_student_confidence_interval(lt_diffs)
    verifica = 'x' if lower_bound <= mean <= upper_bound else ''
    new_data.append({
        "agents_in_the_system": key,
        "lt_difference": mean,
        "lower_bound": lower_bound,
        "upper_bound": upper_bound,
        "verifica": verifica
    })

# Creazione del nuovo DataFrame dalle nuove righe
new_df = pd.DataFrame(new_data)

# Funzione per stampare la somma totale di lt_differences
def print_total_lt_difference(df):
    total_lt_difference = df["lt_difference"].sum()
    print(f"La somma totale di tutti i lt_differences è: {total_lt_difference}")

# Utilizzo della funzione
print_total_lt_difference(new_df)

# Salvataggio del DataFrame in un file CSV
new_df.to_csv("users_times_Config2.csv", index=False)

# Visualizzazione del nuovo DataFrame
print(new_df)

In [7]:
import pandas as pd
import numpy as np
from scipy import stats

# Carica il DataFrame dal file CSV
df = pd.read_csv('./results_summary_convalidation_3.csv')

df = df[df["File"].str.contains("Config4")]

# Converti le colonne in liste di float
df["queueLength_Q1"] = df["queueLength_Q1"].apply(lambda x: list(map(lambda y: int(float(y)), x.split(','))))
df["queueLength_Q2"] = df["queueLength_Q2"].apply(lambda x: list(map(lambda y: int(float(y)), x.split(','))))
df["lifeTime_change_Q1"] = df["lifeTime_change_Q1"].apply(lambda x: list(map(float, x.split(','))))
df["lifeTime_change_Q2"] = df["lifeTime_change_Q2"].apply(lambda x: list(map(float, x.split(','))))

# Dizionario per accumulare i valori di lt_diff per ogni simulazione e numero di utenti
lt_diff_dict = {}

# Itera su tutte le righe del DataFrame filtrato
for index, row in df.iterrows():
    UsersQ1 = row["queueLength_Q1"]
    UsersQ2 = row["queueLength_Q2"]
    TimesQ1 = row["lifeTime_change_Q1"]
    TimesQ2 = row["lifeTime_change_Q2"]

    # Funzione per ottenere il numero di utenti in un determinato momento
    def get_user_count(times, users, time):
        for i, t in enumerate(times):
            if t == time:
                return users[i]
            elif t > time:
                return users[i-1]
        return users[-1]

    # Combiniamo e ordiniamo i tempi
    combined_times = sorted(set(TimesQ1 + TimesQ2))

    # Calcoliamo i valori combinati degli utenti
    UsersQ1_Q2 = []
    for time in combined_times:
        users_q1 = get_user_count(TimesQ1, UsersQ1, time)
        users_q2 = get_user_count(TimesQ2, UsersQ2, time)
        UsersQ1_Q2.append(users_q1 + users_q2)

    # Calcolo delle differenze di tempo (lt_diff) per questa simulazione
    sim_lt_diff = {}
    for i in range(1, len(combined_times)):
        lt_difference = combined_times[i] - combined_times[i-1]
        key = UsersQ1_Q2[i-1]
        if key not in sim_lt_diff:
            sim_lt_diff[key] = 0
        sim_lt_diff[key] += lt_difference

    # Aggiungiamo i lt_diff di questa simulazione al dizionario generale
    for key, value in sim_lt_diff.items():
        if key not in lt_diff_dict:
            lt_diff_dict[key] = []
        lt_diff_dict[key].append(value)

# Funzione per calcolare l'intervallo di confidenza di Student
def t_student_confidence_interval(data, confidence=0.90):
    mean = np.mean(data)
    sem = stats.sem(data)  # Standard error of the mean
    margin = sem * stats.t.ppf((1 + confidence) / 2, len(data) - 1)
    return mean, mean - margin, mean + margin

# Lista per accumulare i dati delle nuove righe
new_data = []

# Calcolo dell'intervallo di confidenza e creazione delle nuove righe
for key, lt_diffs in lt_diff_dict.items():
    mean, lower_bound, upper_bound = t_student_confidence_interval(lt_diffs)
    verifica = 'x' if lower_bound <= mean <= upper_bound else ''
    new_data.append({
        "agents_in_the_system": key,
        "lt_difference": mean,
        "lower_bound": lower_bound,
        "upper_bound": upper_bound,
        "verifica": verifica
    })

# Creazione del nuovo DataFrame dalle nuove righe
new_df = pd.DataFrame(new_data)

# Funzione per stampare la somma totale di lt_differences
def print_total_lt_difference(df):
    total_lt_difference = df["lt_difference"].sum()
    print(f"La somma totale di tutti i lt_differences è: {total_lt_difference}")

# Utilizzo della funzione
print_total_lt_difference(new_df)

# Salvataggio del DataFrame in un file CSV
new_df.to_csv("users_times_Config4.csv", index=False)

# Visualizzazione del nuovo DataFrame
print(new_df)

La somma totale di tutti i lt_differences è: 10016.34482398942
    agents_in_the_system  lt_difference  lower_bound  upper_bound verifica
0                      0    4429.770966  4357.111879  4502.430052        x
1                      1    1756.506426  1734.452486  1778.560366        x
2                      2    1369.987108  1350.798149  1389.176067        x
3                      3     955.754731   935.070644   976.438819        x
4                      4     595.654726   575.500819   615.808633        x
5                      5     355.099441   339.429840   370.769042        x
6                      6     208.558717   194.370585   222.746849        x
7                      7     126.587714   113.881403   139.294025        x
8                      8      76.436567    65.367042    87.506092        x
9                      9      47.837404    38.352879    57.321930        x
10                    10      30.444115    23.330946    37.557285        x
11                    11      17.6857

/home/stintipacchio/miniconda3/envs/inifile/lib/python3.12/site-packages/numpy/_core/_methods.py:218: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/stintipacchio/miniconda3/envs/inifile/lib/python3.12/site-packages/numpy/_core/_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [8]:
import pandas as pd
import numpy as np
from scipy import stats

# Carica il DataFrame dal file CSV
df = pd.read_csv('./results_summary_convalidation_3.csv')

df = df[df["File"].str.contains("Config6")]

# Converti le colonne in liste di float
df["queueLength_Q1"] = df["queueLength_Q1"].apply(lambda x: list(map(lambda y: int(float(y)), x.split(','))))
df["queueLength_Q2"] = df["queueLength_Q2"].apply(lambda x: list(map(lambda y: int(float(y)), x.split(','))))
df["lifeTime_change_Q1"] = df["lifeTime_change_Q1"].apply(lambda x: list(map(float, x.split(','))))
df["lifeTime_change_Q2"] = df["lifeTime_change_Q2"].apply(lambda x: list(map(float, x.split(','))))

# Dizionario per accumulare i valori di lt_diff per ogni simulazione e numero di utenti
lt_diff_dict = {}

# Itera su tutte le righe del DataFrame filtrato
for index, row in df.iterrows():
    UsersQ1 = row["queueLength_Q1"]
    UsersQ2 = row["queueLength_Q2"]
    TimesQ1 = row["lifeTime_change_Q1"]
    TimesQ2 = row["lifeTime_change_Q2"]

    # Funzione per ottenere il numero di utenti in un determinato momento
    def get_user_count(times, users, time):
        for i, t in enumerate(times):
            if t == time:
                return users[i]
            elif t > time:
                return users[i-1]
        return users[-1]

    # Combiniamo e ordiniamo i tempi
    combined_times = sorted(set(TimesQ1 + TimesQ2))

    # Calcoliamo i valori combinati degli utenti
    UsersQ1_Q2 = []
    for time in combined_times:
        users_q1 = get_user_count(TimesQ1, UsersQ1, time)
        users_q2 = get_user_count(TimesQ2, UsersQ2, time)
        UsersQ1_Q2.append(users_q1 + users_q2)

    # Calcolo delle differenze di tempo (lt_diff) per questa simulazione
    sim_lt_diff = {}
    for i in range(1, len(combined_times)):
        lt_difference = combined_times[i] - combined_times[i-1]
        key = UsersQ1_Q2[i-1]
        if key not in sim_lt_diff:
            sim_lt_diff[key] = 0
        sim_lt_diff[key] += lt_difference

    # Aggiungiamo i lt_diff di questa simulazione al dizionario generale
    for key, value in sim_lt_diff.items():
        if key not in lt_diff_dict:
            lt_diff_dict[key] = []
        lt_diff_dict[key].append(value)

# Funzione per calcolare l'intervallo di confidenza di Student
def t_student_confidence_interval(data, confidence=0.90):
    mean = np.mean(data)
    sem = stats.sem(data)  # Standard error of the mean
    margin = sem * stats.t.ppf((1 + confidence) / 2, len(data) - 1)
    return mean, mean - margin, mean + margin

# Lista per accumulare i dati delle nuove righe
new_data = []

# Calcolo dell'intervallo di confidenza e creazione delle nuove righe
for key, lt_diffs in lt_diff_dict.items():
    mean, lower_bound, upper_bound = t_student_confidence_interval(lt_diffs)
    verifica = 'x' if lower_bound <= mean <= upper_bound else ''
    new_data.append({
        "agents_in_the_system": key,
        "lt_difference": mean,
        "lower_bound": lower_bound,
        "upper_bound": upper_bound,
        "verifica": verifica
    })

# Creazione del nuovo DataFrame dalle nuove righe
new_df = pd.DataFrame(new_data)

# Funzione per stampare la somma totale di lt_differences
def print_total_lt_difference(df):
    total_lt_difference = df["lt_difference"].sum()
    print(f"La somma totale di tutti i lt_differences è: {total_lt_difference}")

# Utilizzo della funzione
print_total_lt_difference(new_df)

# Salvataggio del DataFrame in un file CSV
new_df.to_csv("users_times_Config6.csv", index=False)

# Visualizzazione del nuovo DataFrame
print(new_df)

La somma totale di tutti i lt_differences è: 10005.716179339344
    agents_in_the_system  lt_difference  lower_bound  upper_bound verifica
0                      0    5868.233158  5814.081200  5922.385115        x
1                      1    1726.391317  1710.837695  1741.944939        x
2                      2    1137.909612  1120.023911  1155.795313        x
3                      3     661.210396   644.460210   677.960582        x
4                      4     324.527908   310.661485   338.394332        x
5                      5     150.107319   139.644812   160.569826        x
6                      6      71.131172    63.656684    78.605660        x
7                      7      32.978727    28.314999    37.642456        x
8                      8      13.707761    11.015563    16.399958        x
9                      9       6.232831     4.343204     8.122458        x
10                    10       4.060238     2.326424     5.794051        x
11                    11       2.889

In [9]:
import pandas as pd
import numpy as np
from scipy import stats

# Carica il DataFrame dal file CSV
df = pd.read_csv('./results_summary_convalidation_3.csv')

df = df[df["File"].str.contains("Config8")]

# Converti le colonne in liste di float
df["queueLength_Q1"] = df["queueLength_Q1"].apply(lambda x: list(map(lambda y: int(float(y)), x.split(','))))
df["queueLength_Q2"] = df["queueLength_Q2"].apply(lambda x: list(map(lambda y: int(float(y)), x.split(','))))
df["lifeTime_change_Q1"] = df["lifeTime_change_Q1"].apply(lambda x: list(map(float, x.split(','))))
df["lifeTime_change_Q2"] = df["lifeTime_change_Q2"].apply(lambda x: list(map(float, x.split(','))))

# Dizionario per accumulare i valori di lt_diff per ogni simulazione e numero di utenti
lt_diff_dict = {}

# Itera su tutte le righe del DataFrame filtrato
for index, row in df.iterrows():
    UsersQ1 = row["queueLength_Q1"]
    UsersQ2 = row["queueLength_Q2"]
    TimesQ1 = row["lifeTime_change_Q1"]
    TimesQ2 = row["lifeTime_change_Q2"]

    # Funzione per ottenere il numero di utenti in un determinato momento
    def get_user_count(times, users, time):
        for i, t in enumerate(times):
            if t == time:
                return users[i]
            elif t > time:
                return users[i-1]
        return users[-1]

    # Combiniamo e ordiniamo i tempi
    combined_times = sorted(set(TimesQ1 + TimesQ2))

    # Calcoliamo i valori combinati degli utenti
    UsersQ1_Q2 = []
    for time in combined_times:
        users_q1 = get_user_count(TimesQ1, UsersQ1, time)
        users_q2 = get_user_count(TimesQ2, UsersQ2, time)
        UsersQ1_Q2.append(users_q1 + users_q2)

    # Calcolo delle differenze di tempo (lt_diff) per questa simulazione
    sim_lt_diff = {}
    for i in range(1, len(combined_times)):
        lt_difference = combined_times[i] - combined_times[i-1]
        key = UsersQ1_Q2[i-1]
        if key not in sim_lt_diff:
            sim_lt_diff[key] = 0
        sim_lt_diff[key] += lt_difference

    # Aggiungiamo i lt_diff di questa simulazione al dizionario generale
    for key, value in sim_lt_diff.items():
        if key not in lt_diff_dict:
            lt_diff_dict[key] = []
        lt_diff_dict[key].append(value)

# Funzione per calcolare l'intervallo di confidenza di Student
def t_student_confidence_interval(data, confidence=0.90):
    mean = np.mean(data)
    sem = stats.sem(data)  # Standard error of the mean
    margin = sem * stats.t.ppf((1 + confidence) / 2, len(data) - 1)
    return mean, mean - margin, mean + margin

# Lista per accumulare i dati delle nuove righe
new_data = []

# Calcolo dell'intervallo di confidenza e creazione delle nuove righe
for key, lt_diffs in lt_diff_dict.items():
    mean, lower_bound, upper_bound = t_student_confidence_interval(lt_diffs)
    verifica = 'x' if lower_bound <= mean <= upper_bound else ''
    new_data.append({
        "agents_in_the_system": key,
        "lt_difference": mean,
        "lower_bound": lower_bound,
        "upper_bound": upper_bound,
        "verifica": verifica
    })

# Creazione del nuovo DataFrame dalle nuove righe
new_df = pd.DataFrame(new_data)

# Funzione per stampare la somma totale di lt_differences
def print_total_lt_difference(df):
    total_lt_difference = df["lt_difference"].sum()
    print(f"La somma totale di tutti i lt_differences è: {total_lt_difference}")

# Utilizzo della funzione
print_total_lt_difference(new_df)

# Salvataggio del DataFrame in un file CSV
new_df.to_csv("users_times_Config8.csv", index=False)

# Visualizzazione del nuovo DataFrame
print(new_df)

La somma totale di tutti i lt_differences è: 10007.804964013245
    agents_in_the_system  lt_difference  lower_bound  upper_bound verifica
0                      0    6899.833141  6855.577762  6944.088520        x
1                      1    1514.747797  1499.437699  1530.057895        x
2                      2     874.815177   859.952477   889.677876        x
3                      3     437.433141   424.362161   450.504120        x
4                      4     174.323865   164.927974   183.719756        x
5                      5      62.599899    57.737008    67.462790        x
6                      6      22.318745    19.855750    24.781740        x
7                      7       7.108501     5.568115     8.648887        x
8                      8       3.036412     2.024410     4.048413        x
9                      9       1.718821     0.869090     2.568553        x
10                    10       1.341263     0.132039     2.550488        x
11                    11       2.057

/home/stintipacchio/miniconda3/envs/inifile/lib/python3.12/site-packages/numpy/_core/_methods.py:218: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/stintipacchio/miniconda3/envs/inifile/lib/python3.12/site-packages/numpy/_core/_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [10]:
import pandas as pd
import numpy as np
from scipy import stats

# Carica il DataFrame dal file CSV
df = pd.read_csv('./results_summary_convalidation_3.csv')

df = df[df["File"].str.contains("Config10")]

# Converti le colonne in liste di float
df["queueLength_Q1"] = df["queueLength_Q1"].apply(lambda x: list(map(lambda y: int(float(y)), x.split(','))))
df["queueLength_Q2"] = df["queueLength_Q2"].apply(lambda x: list(map(lambda y: int(float(y)), x.split(','))))
df["lifeTime_change_Q1"] = df["lifeTime_change_Q1"].apply(lambda x: list(map(float, x.split(','))))
df["lifeTime_change_Q2"] = df["lifeTime_change_Q2"].apply(lambda x: list(map(float, x.split(','))))

# Dizionario per accumulare i valori di lt_diff per ogni simulazione e numero di utenti
lt_diff_dict = {}

# Itera su tutte le righe del DataFrame filtrato
for index, row in df.iterrows():
    UsersQ1 = row["queueLength_Q1"]
    UsersQ2 = row["queueLength_Q2"]
    TimesQ1 = row["lifeTime_change_Q1"]
    TimesQ2 = row["lifeTime_change_Q2"]

    # Funzione per ottenere il numero di utenti in un determinato momento
    def get_user_count(times, users, time):
        for i, t in enumerate(times):
            if t == time:
                return users[i]
            elif t > time:
                return users[i-1]
        return users[-1]

    # Combiniamo e ordiniamo i tempi
    combined_times = sorted(set(TimesQ1 + TimesQ2))

    # Calcoliamo i valori combinati degli utenti
    UsersQ1_Q2 = []
    for time in combined_times:
        users_q1 = get_user_count(TimesQ1, UsersQ1, time)
        users_q2 = get_user_count(TimesQ2, UsersQ2, time)
        UsersQ1_Q2.append(users_q1 + users_q2)

    # Calcolo delle differenze di tempo (lt_diff) per questa simulazione
    sim_lt_diff = {}
    for i in range(1, len(combined_times)):
        lt_difference = combined_times[i] - combined_times[i-1]
        key = UsersQ1_Q2[i-1]
        if key not in sim_lt_diff:
            sim_lt_diff[key] = 0
        sim_lt_diff[key] += lt_difference

    # Aggiungiamo i lt_diff di questa simulazione al dizionario generale
    for key, value in sim_lt_diff.items():
        if key not in lt_diff_dict:
            lt_diff_dict[key] = []
        lt_diff_dict[key].append(value)

# Funzione per calcolare l'intervallo di confidenza di Student
def t_student_confidence_interval(data, confidence=0.90):
    mean = np.mean(data)
    sem = stats.sem(data)  # Standard error of the mean
    margin = sem * stats.t.ppf((1 + confidence) / 2, len(data) - 1)
    return mean, mean - margin, mean + margin

# Lista per accumulare i dati delle nuove righe
new_data = []

# Calcolo dell'intervallo di confidenza e creazione delle nuove righe
for key, lt_diffs in lt_diff_dict.items():
    mean, lower_bound, upper_bound = t_student_confidence_interval(lt_diffs)
    verifica = 'x' if lower_bound <= mean <= upper_bound else ''
    new_data.append({
        "agents_in_the_system": key,
        "lt_difference": mean,
        "lower_bound": lower_bound,
        "upper_bound": upper_bound,
        "verifica": verifica
    })

# Creazione del nuovo DataFrame dalle nuove righe
new_df = pd.DataFrame(new_data)

# Funzione per stampare la somma totale di lt_differences
def print_total_lt_difference(df):
    total_lt_difference = df["lt_difference"].sum()
    print(f"La somma totale di tutti i lt_differences è: {total_lt_difference}")

# Utilizzo della funzione
print_total_lt_difference(new_df)

# Salvataggio del DataFrame in un file CSV
new_df.to_csv("users_times_Config10.csv", index=False)

# Visualizzazione del nuovo DataFrame
print(new_df)

La somma totale di tutti i lt_differences è: 10025.913362364214
    agents_in_the_system  lt_difference  lower_bound  upper_bound verifica
0                      0    4547.550510  4472.171833  4622.929187        x
1                      1    1857.657594  1832.744323  1882.570866        x
2                      2    1361.291469  1340.325237  1382.257701        x
3                      3     882.316432   862.765287   901.867577        x
4                      4     542.175910   521.779301   562.572519        x
5                      5     316.166680   295.044875   337.288484        x
6                      6     187.800743   171.909400   203.692086        x
7                      7     114.615350   101.075017   128.155682        x
8                      8      73.337905    62.178486    84.497325        x
9                      9      45.327510    36.578538    54.076482        x
10                    10      27.183084    20.577940    33.788228        x
11                    11      15.255